In [41]:
import pandas as pd
from tqdm import tqdm
import json
import os

import re

In [42]:
file_path = '../../dataset/slurp/dataset/slurp/'

#'transcript','intent','action','paths', 'entities'
with open(os.path.join(file_path,'train.jsonl')) as f:
    entries = []
    entity_type = []
    for i, line in tqdm(enumerate(f)):
        if i < 0: break
        line = json.loads(line)
        temp =[]
        temp.append(line['sentence'])
        temp.append(line['intent'])
        temp.append(line['action'])
        temp.append([path['file'] for path in line['recordings']])
        entities = ['O' for _ in range(len(re.split(r"[ \-\']",line['sentence'])))]
        for entity in line['entities']:
            for ind in entity['span']:
                    entities[ind] = entity['type']
                    if entity['type'] not in entity_type:
                         entity_type.append(entity['type'])
        temp.append(entities)
        entries.append(temp)

df = pd.DataFrame(entries,columns=['transcript','intent','action','paths', 'entities'])
df.head()


11513it [00:00, 34122.10it/s]


,transcript,intent,action,paths,entities
0,event,calendar_set,set,"[audio-1501754435.flac, audio-1501407267-heads...",[O]
1,i need an event three days from now scheduled ...,calendar_set,set,"[audio-1498578293.flac, audio-1490262086-heads...","[O, O, O, O, date, date, date, date, O, O, per..."
2,turn up,audio_volume_up,volume_up,"[audio-1490183568-headset.flac, audio-14901835...","[O, O]"
3,brighten the lights a little bit,iot_hue_lightup,hue_lightup,"[audio-1490019123-headset.flac, audio-14900191...","[O, O, O, O, O, O]"
4,what's the current weather,weather_query,query,[audio-1501754622.flac],"[O, O, O, O, O]"


In [43]:
len(entity_type)

55

In [44]:
df.to_csv(os.path.join(file_path,'train_data.csv'))


In [45]:
# import torchaudio 
# from IPython.display import Audio

# signal,rate = torchaudio.load('../../dataset/slurp/scripts/audio/slurp_real/audio-1500979433.flac')
# Audio(signal,rate=rate)


In [1]:
import os
import pandas as pd
from tqdm import tqdm


dataset_path_base = '../../dataset/fluent_speech_commands_dataset/'

def prep_data(train_df):
    transcription_data = []
    for i in tqdm(range(0,len(train_df))):
        path = train_df.loc[i,'path']
        true_transcription = train_df.loc[i,'transcription']
        action = train_df.loc[i,'action']
        object = train_df.loc[i,'object']
        location = train_df.loc[i,'location']
        labels = []
        for v in true_transcription.split(" "):
            if(v == object):
                labels.append('object')
            elif(v == location):
                labels.append('location')
            else:
                labels.append('O')
        transcription_data.append([path,true_transcription,action,','.join(labels)])

    transcription_df = pd.DataFrame(transcription_data,columns=['path','transcription','action','labels'])
    return transcription_df




train_df = pd.read_csv(os.path.join(dataset_path_base+'/data','train_data.csv'))
transcription_df = prep_data(train_df)
transcription_df.to_csv(os.path.join(dataset_path_base+'/data','train_labeled.csv'), index=False)

valid_df = pd.read_csv(os.path.join(dataset_path_base+'/data','valid_data.csv'))
transcription_df = prep_data(valid_df)
transcription_df.to_csv(os.path.join(dataset_path_base+'/data','valid_labeled.csv'), index=False)

test_df = pd.read_csv(os.path.join(dataset_path_base+'/data','test_data.csv'))
transcription_df = prep_data(test_df)
transcription_df.to_csv(os.path.join(dataset_path_base+'/data','test_labeled.csv'), index=False)

100%|██████████| 3793/3793 [00:00<00:00, 32098.73it/s]


In [4]:
import torch

# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    # Get the number of available CUDA devices
    num_cuda_devices = torch.cuda.device_count()
    print(num_cuda_devices)
    # Loop through each CUDA device
    for device_idx in range(num_cuda_devices):
        device = torch.device(f'cuda:{device_idx}')
        device_name = torch.cuda.get_device_name(device)
        print(f"Device {device_idx}: {device_name}")

        # Get device's memory information
        total_memory = torch.cuda.get_device_properties(device).total_memory
        free_memory = torch.cuda.memory_reserved(device) - torch.cuda.memory_allocated(device)
        used_memory = torch.cuda.memory_allocated(device)

        # Convert memory values to human-readable format
        total_memory = total_memory / (1024**3)  # Convert bytes to gigabytes
        free_memory = free_memory / (1024**2)
        used_memory = used_memory / (1024**3)

        print(f"Total Memory: {total_memory:.2f} GB")
        print(f"Used Memory: {used_memory:.2f} GB")
        print(f"Free Memory: {free_memory:.2f} MB")
        print("-" * 40)
else:
    print('No CUDA devices (GPUs) available')


8
Device 0: Tesla V100-SXM2-32GB
Total Memory: 31.75 GB
Used Memory: 0.00 GB
Free Memory: 0.00 MB
----------------------------------------
Device 1: Tesla V100-SXM2-32GB
Total Memory: 31.75 GB
Used Memory: 0.00 GB
Free Memory: 0.00 MB
----------------------------------------
Device 2: Tesla V100-SXM2-32GB
Total Memory: 31.75 GB
Used Memory: 0.00 GB
Free Memory: 0.00 MB
----------------------------------------
Device 3: Tesla V100-SXM2-32GB
Total Memory: 31.75 GB
Used Memory: 0.00 GB
Free Memory: 0.00 MB
----------------------------------------
Device 4: Tesla V100-SXM2-32GB
Total Memory: 31.75 GB
Used Memory: 0.00 GB
Free Memory: 0.00 MB
----------------------------------------
Device 5: Tesla V100-SXM2-32GB
Total Memory: 31.75 GB
Used Memory: 0.00 GB
Free Memory: 0.00 MB
----------------------------------------
Device 6: Tesla V100-SXM2-32GB
Total Memory: 31.75 GB
Used Memory: 0.00 GB
Free Memory: 0.00 MB
----------------------------------------
Device 7: Tesla V100-SXM2-32GB
Total Me